In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os, sys
sys.path.append("/home/suman/Sayari/TrP-cage/Noteboks_Trpcage")
#import modules
import numpy as np
import MDAnalysis as mda
from sklearn.preprocessing import MinMaxScaler
import pyemma
import matplotlib.pyplot as plt
##custom scripts
from runner import Feature
from plot2d import plot_free_energy as pfe


In [ ]:
pwd

# Reading Trajectory

In [ ]:
## De Shaw TRP-Cage trajectory
path = "/home/suman/Dibyendu/Trp-Cage-folding/DESRES-Trajectory_2JOF-0-protein/2JOF-0-protein/"
files = os.listdir(path)
traj_list = []
for file in files:
    if file.endswith(".dcd"):
        traj_list.append(path + file)

sorted_traj_list = list(np.sort(traj_list))
trp_cage = mda.Universe("/home/suman/Sayari/TrP-cage/trp_cage.pdb", sorted_traj_list )

# Input Features

In [ ]:
## Features...give the important contact pairs that have discriminative powers..
ca_dist= np.loadtxt("pairwise_dist.txt")

In [ ]:
#If you used other features calculate them accordingly and then scale them

In [ ]:
## Scaling of data..
scaler = MinMaxScaler()
scaler.fit(ca_dist)
scaled_dist = scaler.transform(ca_dist)

# TICA

In [ ]:
Distances_tica=scaled_dist.reshape(1,1044000,190)
tica = pyemma.coordinates.tica(list(Distances_tica[::]), lag=250) #lagtime 50ns

In [ ]:
tica_output = tica.get_output()
tica_concatenated = np.concatenate(tica_output)

In [ ]:
tica_concatenated.shape

# HDBSCAN

In [ ]:
import cuml # install gpu based hdbscan module cuml

In [ ]:
clusterer_distance = cuml.cluster.hdbscan.HDBSCAN(min_cluster_size=20000,min_samples=200,
                                                  prediction_data=True, gen_min_span_tree=True,cluster_selection_method="eom")
clusterer_distance.fit(scaled_dist[::]) # give input feature of your interest in .fit function, stride if required

In [ ]:
# Min_Samples selection should be choosen based on dimensionality of input feature, it controls the amount of noise data points.
# min_cluster_size is the control parameter for determinimg number of clusters

# extract cluster labels

In [ ]:
clusterlabels= clusterer_distance.labels_
np.unique(clusterlabels)

# Calculate Centriod of Largest Cluster

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
# Find the largest cluster
unique_labels, counts = np.unique(clusterlabels[clusterlabels != -1], return_counts=True)
largest_cluster_label = unique_labels[np.argmax(counts)]

In [ ]:
# Filter points belonging to the largest cluster
largest_cluster = scaled_dist[clusterlabels == largest_cluster_label]

# Calculate the centroid
centroid = np.mean(largest_cluster, axis=0)

In [ ]:
Tica_centriod= tica.transform((centroid.reshape(1, 190)))

# Calculate Centriod of all three clusters

In [ ]:
# Calculate centroids and their tICA transformations for each cluster
centroids = []
tica_centroids = []

for i in range(len(unique_labels)):
    cluster = scaled_dist[clusterlabels == i]
    centroid = np.mean(cluster, axis=0)
    centroids.append(centroid)
    tica_centroids.append(tica.transform(centroid.reshape(1, -1)))


In [ ]:
fig ,ax = plt.subplots(figsize=[10,8])
pfe(tica_concatenated[:,0],tica_concatenated[:,1],ax=ax)
plt.scatter(np.array(tica_centroids)[0][:,0],np.array(tica_centroids)[0][:,1],s=500 ,c='red',marker='*')
plt.scatter(np.array(tica_centroids)[1][:,0],np.array(tica_centroids)[1][:,1],s=500 ,c='red',marker='*')
plt.scatter(np.array(tica_centroids)[2][:,0],np.array(tica_centroids)[2][:,1],s=500 ,c='red',marker='*',label='Cluster Centriods')
plt.legend(loc='upper left',fontsize=20)

In [ ]:
# HDBSCAN better for finding the multiple small clusters in data
# GMM best in separating folded state from the unfolded state

# Gaussian Mixture

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=3, random_state=42,covariance_type='full').fit(scaled_dist[::10])
labels_gm = gm.predict(scaled_dist[::10])

In [ ]:
import scipy.cluster.hierarchy as shc
from scipy.cluster import hierarchy

In [ ]:
import numba as nb
class Hierarchical_Clastering:
    def __init__(self):
        pass
    def rgb_hex(self, color):
        '''converts a (r,g,b) color (either 0-1 or 0-255) to its hex representation.
        for ambiguous pure combinations of 0s and 1s e,g, (0,0,1), (1/1/1) is assumed.'''
        message='color must be an iterable of length 3.'
        assert hasattr(color, '__iter__'), message
        assert len(color)==3, message
        if all((c <= 1) & (c >= 0) for c in color): color=[int(round(c*255)) for c in color] # in case provided rgb is 0-1
        color=tuple(color)
        return '#%02x%02x%02x' % color

    def get_cluster_colors(self, n_clusters, alpha=0.8, alpha_outliers=0.05):
        #my_set_of_20_rgb_colors =
        cluster_colors = [
            [
                np.random.randint(255),
                np.random.randint(255),
                np.random.randint(255),
            ]
            for _ in range(100)
        ]
        cluster_colors = [c+[alpha] for c in cluster_colors]
        outlier_color = [0,0,0,alpha_outliers]
        return [cluster_colors[i%19] for i in range(n_clusters)] + [outlier_color]

    def clusters(self, X, threshold,no_plot = True, method='ward', metric='euclidean', default_color='black'):

        # perform hierarchical clustering
        Z              = hierarchy.linkage(X.astype(np.float32), method=method, metric=metric)

        # get cluster labels
        labels         = hierarchy.fcluster(Z, threshold, criterion='distance') - 1
        labels_str     = [f"cluster #{l}: n={c}\n" for (l,c) in zip(*np.unique(labels, return_counts=True))]
        n_clusters     = len(labels_str)

        cluster_colors = [self.rgb_hex(c[:-1]) for c in self.get_cluster_colors(n_clusters, alpha=0.8, alpha_outliers=0.05)]
        cluster_colors = ["blue" for i in range(n_clusters)]
        #cluster_colors = ["blue", "mediumorchid", "green"] # choose color accordingly

        cluster_colors_array = [cluster_colors[l] for l in labels]
        link_cols = {}
        for i, i12 in enumerate(Z[:,:2].astype(int)):
            c1, c2 = (link_cols[x] if x > len(Z) else cluster_colors_array[x] for x in i12)
            link_cols[i+1+len(Z)] = c1 if c1 == c2 else 'k'

        # plot dendrogram with colored clusters
        if no_plot:
            self._extracted_from_clusters_20(threshold)
        # plot dendrogram based on clustering results
        dend = hierarchy.dendrogram(
            Z,
            no_plot = not no_plot,
            labels = labels,
            color_threshold=threshold,
            truncate_mode = 'level',
            p = 5,
            show_leaf_counts = True,
            leaf_rotation=90,
            leaf_font_size=10,
            show_contracted=False,
            link_color_func=lambda x: link_cols[x],
            above_threshold_color=default_color,
            distance_sort='descending',
            )


        self.labels = labels
        self.dendogram = dend

    # TODO Rename this here and in `clusters`
    def _extracted_from_clusters_20(self, threshold):
        fig, ax = plt.subplots(figsize=(12, 7))
        plt.title('Hierarchical Clustering Dendrogram',loc='center',fontsize=40)
        plt.xlabel('Data points',fontsize=25)
        plt.ylabel('Distance',fontsize=25)
        plt.axhline(threshold, color='gray',lw =2, ls  = "--")
        fig.patch.set_facecolor('white')
        #plt.xticks([])
        #plt.yticks([])

In [ ]:
HC = Hierarchical_Clastering()
HC.clusters(scaled_dist[::50], 120, no_plot=True) #Thresold is dependent on number of data points,
#**HC is the slowest among all**